In [1]:
import base64
import gymnasium as gym
import imageio
from IPython.display import HTML
import numpy as np
import tensorflow as tf

VIDEO_FILEPATH = "./output/lunar_lander.mp4"

In [2]:
env = gym.make('LunarLander-v3', render_mode='rgb_array')
q_func_model = tf.keras.models.load_model("output/lunar_lander.keras")
q_func_model.summary()

LUNAR LANDER MONITOR PROPERTIES:

	 Q-func Model Path: 		./output/lunar_lander.keras
	 Training Data Path: 		./output/latest_buffer_states.npy

	 Output Video Path: 		./output/lunar_lander.mp4
	 Output Video FPS: 			30

	 Monitor Trace Path: 		./output/latest_traces.npy
	 Model Prediction Path: 	./output/latest_preds.npy


In [3]:
with imageio.get_writer(VIDEO_FILEPATH, fps=30) as video:
    done = False
    state = env.reset()[0]
    frame = env.render()
    video.append_data(frame)

    while not done:
        state = np.expand_dims(state, axis=0)
        q_values = q_func_model(state)
        action = np.argmax(q_values.numpy()[0])
        state, _, done, _, _ = env.step(action)
        frame = env.render()
        video.append_data(frame)
    # end while-loop
# end with-block

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden1 (Dense)                 │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ monitor (Dense)                 │ (None, 8)              │           520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 4)              │            36 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,292 (20.67 KB)

 Trainable params: 5,292 (20.67 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
video = open(VIDEO_FILEPATH, "rb").read()
b64 = base64.b64encode(video)
tag = """<video width="608" height="400" controls loop autoplay="true">
        <source src="./output/lunar_lander.mp4" type="video/mp4">
        Your browser does not support the video tag.
        </video>""".format(b64.decode())
HTML(tag)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


In [5]:
states = np.load("./output/latest_buffer_states.npy", allow_pickle=True)
input_data = np.expand_dims(states[0], axis=0)

monitor_layer = q_func_model.get_layer("monitor")
monitor_model = tf.keras.models.Model(inputs=q_func_model.inputs,
                                      outputs=monitor_layer.output)
monitor_trace = monitor_model.predict(input_data)

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 1s 158us/step
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 1s 161us/step

Saved traces with shape: (100000, 8)
Saved predictions with shape: (100000, 4)
